Så først og fremmest få prio grid og UCDP merged og se hvad su så synes du mangler...

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import time

Gelman siger (måske): divider dine continuerlige variabler med 2 std_d for at de kommer til at ligene dine katagoriske var mere.... (https://www.youtube.com/watch?v=Jb9eklfbDyg 16.30)

In [2]:
def my_import():
    """Function for loading, handling and merging the data. Rigth now you don't use the prio gird thou.."""
    t1 = time.time()
    
    # load data:
    prio_grid = gpd.read_file("zip:///home/polichinel/Dropbox/KU/9.semester/friopgave/data/prio_grid/priogrid_cellshp.zip")
    prio_static = pd.read_csv("/home/polichinel/Dropbox/KU/9.semester/friopgave/data/prio_grid/static/PRIO-GRID Static Variables - 2018-08-21.csv")
    prio_yearly_interp = pd.read_csv("/home/polichinel/Dropbox/KU/9.semester/friopgave/data/prio_grid/yearly/PRIO_Yearly_interp.csv")
    prio_yearly = pd.read_csv("/home/polichinel/Dropbox/KU/9.semester/friopgave/data/prio_grid/yearly/PRIO-GRID Yearly Variables for 1946-2014 - 2018-08-21.csv")
    ucdp = pd.read_csv("/home/polichinel/Dropbox/KU/9.semester/friopgave/data/upsala/ged181.csv")# the error is not importent as you do not use that feature 
    
    
    # Choosing relevant variables:
    prio_grid_lean = prio_grid[['gid','geometry']]
    prio_static_lean = prio_static[['gid','landarea','ttime_mean','mountains_mean','petroleum_s']]
    prio_yearly_interp_lean = prio_yearly_interp[['gid','year','interp_pop_gpw_sum','interp_gcp_ppp','interp_urban_ih','interp_agri_ih','interp_forest_ih']]
    prio_yearly_lean = prio_yearly[['gid','year','gwno','gwarea','bdist1','bdist2','bdist3','capdist','excluded','petroleum_y','nlights_calib_mean']] #bdist 3 er minimere nans
    ucdp_lean = ucdp[['priogrid_gid','country_id','region','year','best']]

    
    # Choosing relevant years: 
    prio_yearly_interp_lean_1990 = prio_yearly_interp_lean[(prio_yearly_interp_lean['year'] >= 1990) & (prio_yearly_interp_lean['year'] <= 2010)]
    prio_yearly_lean_1990 = prio_yearly_lean[(prio_yearly_lean['year'] >= 1990) & (prio_yearly_lean['year'] <= 2010)]
    ucdp_lean_1990 = ucdp_lean[(ucdp_lean['year'] >= 1990) & (ucdp_lean['year'] <= 2010)]
    
    # Aggregating yearly fatalities:
    ucdp_lean_1990_year_agg = ucdp_lean_1990.groupby(by=['priogrid_gid','country_id','year']).sum().reset_index() # det her virker.tror jeg
    ucdp_lean_1990_year_agg.rename(columns={'priogrid_gid':'gid'}, inplace=True)
    ucdp_lean_1990_year_agg.rename(columns={'country_id':'gwno'}, inplace=True)
    
    
    # Merging
    prio_yearly_lean_1990_all = pd.merge(prio_yearly_interp_lean_1990, prio_yearly_lean_1990, on = ['gid','year'])

    prio_static_grid = prio_grid_lean.merge(prio_static_lean, on='gid', how='inner')
    prio_sy_grid = prio_static_grid.merge(prio_yearly_lean_1990_all, on = 'gid', how='inner')
    prio_sy_ucdp_grid = prio_sy_grid.merge(ucdp_lean_1990_year_agg, on = ['gid','year','gwno'], how='left')

    
    # Filling in cells with non-fatalities.
    prio_sy_ucdp_grid['best'].fillna(value=0, inplace = True)
    prio_sy_ucdp_grid['log_best'] = np.log(prio_sy_ucdp_grid['best']+1)
    
    # Managing petolium:
    prio_sy_ucdp_grid['petroleum_y'] = prio_sy_ucdp_grid.sort_values('year').groupby('gid')['petroleum_y'].fillna(method = 'ffill').copy() # fint
    prio_sy_ucdp_grid['petroleum_y'].fillna(0, inplace = True)
    prio_sy_ucdp_grid['petroleum_s'].fillna(0, inplace = True)
    
    prio_sy_ucdp_grid['petroleum_full'] = np.maximum(prio_sy_ucdp_grid['petroleum_y'],prio_sy_ucdp_grid['petroleum_s'])
    
    prio_sy_ucdp_grid.drop('petroleum_y', axis=1, inplace=True)
    prio_sy_ucdp_grid.drop('petroleum_s', axis=1, inplace=True)
                                                        
    # Managing bdist1 
    #prio_sy_ucdp_grid['bdist1'].fillna(np.median(prio_sy_ucdp_grid['bdist1']), inplace = True) # kunne være fedt bare at ordne det i modellen.
    
    
    print('seconds it took load and handle data: ' + str(time.time()-t1))
    return(prio_sy_ucdp_grid)
    
    # Should you normelize the data... espec teh y 'best'

In [3]:
df=my_import()
df_s = df.sample(10000, random_state=123)

# feature_list = ['landarea', 'ttime_mean', 'mountains_mean',
#        'petroleum_s', 'year', 'interp_pop_gpw_sum', 'interp_gcp_ppp',
#        'interp_urban_ih', 'interp_agri_ih', 'interp_forest_ih', 'gwno',
#        'gwarea', 'bdist1', 'capdist', 'excluded', 'petroleum_y',
#        'nlights_calib_mean', 'best']

feature_list = ['landarea', 'ttime_mean', 'mountains_mean',
       'petroleum_full', 'year', 'interp_pop_gpw_sum', 'interp_gcp_ppp',
       'interp_urban_ih', 'interp_agri_ih', 'interp_forest_ih', 'gwno',
       'gwarea','bdist1','bdist2', 'bdist3', 'capdist', 'excluded', 'nlights_calib_mean', 'best']

df.shape


/home/polichinel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


seconds it took load and handle data: 66.70944619178772


(1361178, 22)

In [4]:
def get_na(df=df):

    na_list = []
    
    for i in df.columns:
        proporiton_na = df[i].isna().sum()/len(df[i])
        print(i + " that is NA : " + str(round(proporiton_na*100, 3)) + '%')
        
        
        
        if proporiton_na > 0.01:
            na_list.append(i)
        
    return(na_list)

na_list = get_na()

gid that is NA : 0.0%
geometry that is NA : 0.0%
landarea that is NA : 0.0%
ttime_mean that is NA : 0.156%
mountains_mean that is NA : 4.374%
year that is NA : 0.0%
interp_pop_gpw_sum that is NA : 0.0%
interp_gcp_ppp that is NA : 26.817%
interp_urban_ih that is NA : 7.509%
interp_agri_ih that is NA : 7.509%
interp_forest_ih that is NA : 7.509%
gwno that is NA : 0.001%
gwarea that is NA : 0.001%
bdist1 that is NA : 7.024%
bdist2 that is NA : 0.001%
bdist3 that is NA : 0.001%
capdist that is NA : 0.001%
excluded that is NA : 20.131%
nlights_calib_mean that is NA : 12.73%
best that is NA : 0.0%
log_best that is NA : 0.0%
petroleum_full that is NA : 0.0%


In [5]:
# #uncomment to visualize all nans:

# for y in np.sort(df['year'].unique()):    
#     for f in na_list:
#         #if df[f].isna().any():
#         if df[str(f)].dtype != type(object):

#             df[(df['year']==y) & (df[f].isna())].plot(column = f, figsize=(15, 15))
#             plt.title(f + '-' + str(y) + '_nan_cells')
#             plt.show()
        

In [6]:
def fill_missing_1(df=df):
    
    # First back-extrapolating light values from 1992 to 91 and 90.
    
    Light_years = [1990, 1991]# full missing years for 'nlights_calib_mean'.
    missing_list = ['interp_urban_ih', 'interp_agri_ih', 'interp_forest_ih','mountains_mean','nlights_calib_mean']
    
    for y in Light_years:
        df.loc[df['year'] == y, 'nlights_calib_mean'] = df[df['year'] == 1992]['nlights_calib_mean'].values


    for f in missing_list:
        df.loc[df[f] < 0 , f] = 0 # due to rounding errors in the interpolation, few values a negative (still practical zero.)
    
        df[f].fillna(df[f].min(), inplace = True) # minimun in this case being zero.

fill_missing_1()
na_list = get_na()

gid that is NA : 0.0%
geometry that is NA : 0.0%
landarea that is NA : 0.0%
ttime_mean that is NA : 0.156%
mountains_mean that is NA : 0.0%
year that is NA : 0.0%
interp_pop_gpw_sum that is NA : 0.0%
interp_gcp_ppp that is NA : 26.817%
interp_urban_ih that is NA : 0.0%
interp_agri_ih that is NA : 0.0%
interp_forest_ih that is NA : 0.0%
gwno that is NA : 0.001%
gwarea that is NA : 0.001%
bdist1 that is NA : 7.024%
bdist2 that is NA : 0.001%
bdist3 that is NA : 0.001%
capdist that is NA : 0.001%
excluded that is NA : 20.131%
nlights_calib_mean that is NA : 0.0%
best that is NA : 0.0%
log_best that is NA : 0.0%
petroleum_full that is NA : 0.0%


In [7]:
# # uncomment to vizualize:

# na_list = ['interp_gcp_ppp', 'bdist1', 'excluded']

# for y in np.sort(df['year'].unique()):    
#     for f in na_list:
#         #if df[f].isna().any():
#         if df[str(f)].dtype != type(object):

#             df[(df['year']==y) & (df[f].isna())].plot(column = f, figsize=(15, 15))
#             plt.title(f + '-' + str(y) + '_nan_cells')
#             plt.show()

In [8]:
def fill_missing_2(df=df):
    
    # first bdist1:
    df['bdist1'].fillna(df['bdist1'].median(), inplace = True) # median and not mean, since it is is skewed - ofc should be corss-validated. 
    
    # Then, correct the interpolation:
    df.loc[df['interp_gcp_ppp'] < 0 , 'interp_gcp_ppp'] = 0 # due to rounding errors in the interpolation, few values a negative (still practical zero.)
    
    # fill forward:
    df['interp_gcp_ppp'] = df.sort_values('year').groupby('gid')['interp_gcp_ppp'].fillna(method = 'ffill')
    df['interp_gcp_ppp'] = df.sort_values('year').groupby('gid')['interp_gcp_ppp'].fillna(method = 'bfill')

    
    # Excluded = min
    df['excluded'].fillna(df['excluded'].min(), inplace = True) # tjek lige i codebogen om det her er khoser
    
    
fill_missing_2()
na_list = get_na()

gid that is NA : 0.0%
geometry that is NA : 0.0%
landarea that is NA : 0.0%
ttime_mean that is NA : 0.156%
mountains_mean that is NA : 0.0%
year that is NA : 0.0%
interp_pop_gpw_sum that is NA : 0.0%
interp_gcp_ppp that is NA : 2.058%
interp_urban_ih that is NA : 0.0%
interp_agri_ih that is NA : 0.0%
interp_forest_ih that is NA : 0.0%
gwno that is NA : 0.001%
gwarea that is NA : 0.001%
bdist1 that is NA : 0.0%
bdist2 that is NA : 0.001%
bdist3 that is NA : 0.001%
capdist that is NA : 0.001%
excluded that is NA : 0.0%
nlights_calib_mean that is NA : 0.0%
best that is NA : 0.0%
log_best that is NA : 0.0%
petroleum_full that is NA : 0.0%


In [9]:
# # uncomment to visualize:

# for y in np.sort(df['year'].unique()):    

#     if df[str('interp_gcp_ppp')].dtype != type(object):

#         df[(df['year']==y) & (df['interp_gcp_ppp'].isna())].plot(column = 'interp_gcp_ppp', figsize=(15, 15))
#         plt.title('interp_gcp_ppp' + '-' + str(y) + '_nan_cells')
#         plt.show()